In [33]:
from pydantic import Field, BaseModel
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## Langsmith Tracking And Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"


In [34]:
class Product(BaseModel):
    name:str=Field(description="Provide actual name of the product")
    details: str = Field(max_length=200,description="A small summary of the product")
    price: int = Field(description="provide tentative price of the product in USDwith doller sign")
    

In [35]:
from langchain_groq import ChatGroq
model=ChatGroq(model="qwen-qwq-32b")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x118b47e60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x118b4e690>, model_name='qwen-qwq-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [36]:
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser,PydanticOutputParser
output_parser=JsonOutputParser(pydantic_object=Product)
# format_instructions =  output_parser.get_format_instructions()



In [37]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate(
    [
    ("system", "You are a sales helpful assistant. provide details for asked products in json{format_instruction}"),
    ("human","{product}"),
    
    ]
)

prompt1=prompt.partial(format_instruction=output_parser.get_format_instructions())

In [38]:
output_parser

JsonOutputParser(pydantic_object=<class '__main__.Product'>)

In [39]:
chain=prompt1|model|output_parser
chain

ChatPromptTemplate(input_variables=['product'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Provide actual name of the product", "title": "Name", "type": "string"}, "details": {"description": "A small summary of the product", "maxLength": 200, "title": "Details", "type": "string"}, "price": {"description": "provide tentative price of the product in USDwith doller sign", "title": "Price", "type": "integer"}}, "required": ["name", "details", "price"]}\n```'}, messages=[SystemMessagePr

In [40]:
print(chain.invoke({"product":"iPhone 15"}))


{'name': 'iPhone 15', 'details': 'The iPhone 15 features a durable titanium frame, a 48MP main camera with improved low-light performance, and introduces new color options like Blue. Powered by the A17 Pro chip, it delivers enhanced performance and efficiency.', 'price': 799}
